In [1]:
#train the baseline CNN model for MNIST. Model is identical to that used in the tensorflow
#tutorial: https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html

import mnist_input
import mnist_model
reload(mnist_model)
import numpy as np
import tensorflow as tf
mnist = mnist_input.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
checkpoint_path = "/home/justin/Programming/AdverserialMNIST/saved_models/baseline.ckpt"
x, y_ = mnist_model.place_holders()
y_conv, keep_prob, variable_dict = mnist_model.model(x)

In [3]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv)) #loss function

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

saver = tf.train.Saver(variable_dict)
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver.restore(sess, checkpoint_path)

In [11]:
for i in range(15000):
  batch = mnist.train.next_batch(50)
  if i%1000 == 0: #every 1000 batches we save the model and output the train accuracy
    saver.save(sess, checkpoint_path)
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})




step 0, training accuracy 0.98
step 1000, training accuracy 1
step 2000, training accuracy 1
step 3000, training accuracy 1
step 4000, training accuracy 0.98
step 5000, training accuracy 1
step 6000, training accuracy 1
step 7000, training accuracy 1
step 8000, training accuracy 1
step 9000, training accuracy 1
step 10000, training accuracy 1
step 11000, training accuracy 1
step 12000, training accuracy 1
step 13000, training accuracy 1
step 14000, training accuracy 1


In [12]:
#compute accuracy on test set
#my GPU gives a memory error if I try to run accuracy on the whole test set
#so we feed in the test set in batches and aggregate the results
idx = 0
batch_size = 500
num_correct = 0
while(idx < len(mnist.test.images)):
    num_correct += np.sum(correct_prediction.eval(feed_dict = {
           x: mnist.test.images[idx:idx+batch_size], 
           y_: mnist.test.labels[idx:idx+batch_size], keep_prob: 1.0
                }))
    idx+=batch_size
print "test accuracy: %f" %(float(num_correct)/float(len(mnist.test.images)))


test accuracy: 0.991800


In [4]:
#define the l1 norm of all model parameters
absolute_sums = []
for variable in variable_dict.values():
    absolute_sums.append(tf.reduce_sum(tf.abs(variable)))
l1_sum = tf.add_n(absolute_sums)

In [6]:
#total weight of all of the parameters in the model (for comparison with the l1-regularized model)
sess.run(l1_sum)

239372.95